# Workshop Tutorial: Datasets

This tutorial covers PHOEBE datasets, adding them to the Bundle, and available options for light curves, radial velocity curves, orbits, and meshes.

This workshop tutorial covers many of the same topics as the corresponding online tutorials:

* [Datasets](http://phoebe-project.org/docs/2.4/tutorials/datasets.ipynb)
* [Advanced: Datasets (passband options, dealing with phases, removing datasets)](http://phoebe-project.org/docs/2.4/tutorials/datasets_advanced.ipynb)

# Setup

To begin, let us do the usual imports, set up the logger and load the default binary bundle:

In [1]:
import phoebe
from phoebe import u, c

logger = phoebe.logger(clevel='WARNING')
b = phoebe.default_binary()

# Datasets

PHOEBE interfaces data through *datasets*. These datasets can contain actual observations or they can simply parametrize the forward-model observable that PHOEBE can synthesize. The following dataset kinds are currently supported:

* `lc` (light curves)
* `rv` (radial velocitiy curves)
* `lp` (line profiles)
* `orb` (orbits)
* `mesh` (meshes)

In order to compute a synthetic model, we must first add a dataset to the Bundle via the [add_dataset()](http://phoebe-project.org/docs/dev/api/phoebe.frontend.bundle.Bundle.add_dataset.md) method. The first argument is the kind (one among those listed above). Although that is the only *required* argument, we will usually want to provide a list of times at which we want that dataset computed, along with a label for the dataset. If we do not provide a label, the dataset will be labeled automatically by 'kind' + two digit integer, e.g. 'lc01', 'rv02', etc.).

For the purposes of this tutorial, we will use two array-generating functions: [linspace](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) and [arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html). These are numpy functions, but PHOEBE also provides them natively, thus saving memory by storing less within the bundle.

## Light Curves

Adding a single light curve with predefined times and a label is accomplished by calling the `add_dataset()` bundle method:

In [2]:
b.add_dataset('lc', compute_times=phoebe.linspace(0, 1, 51), dataset='lc01')

<ParameterSet: 80 parameters | contexts: figure, constraint, dataset, compute>

This attaches a set of new dataset parameters to the bundle. Most have the context='dataset', but a few have context='compute', 'figure', and 'constraint'. The 'times' parameter is set with the provided array, and all parameters are tagged with dataset='lc01'.

In [3]:
b.filter(dataset='lc01').contexts

['dataset', 'compute', 'constraint', 'figure']

We can now take a closer look at the new parameters with context='dataset':

In [4]:
b.filter(dataset='lc01', context='dataset').qualifiers

['times',
 'fluxes',
 'passband',
 'intens_weighting',
 'compute_times',
 'compute_phases',
 'phases_t0',
 'mask_enabled',
 'mask_phases',
 'solver_times',
 'sigmas',
 'pblum_mode',
 'pblum_component',
 'l3_mode',
 'l3',
 'exptime',
 'ld_mode',
 'pblum']

These parameters describe *observables* (`times`, `fluxes`, `sigmas`), *passband information* (`passband`, `pblum_mode`, `pblum_component`), *limb darkening* (`ld_mode`), *instrumental information* (`exptime`, `l3_mode`, `l3`, `intens_weighting`), phasing (`phases_t0`) and masking the data (`mask_enabled`, `mask_phases`).

You may have noticed that we used the `compute_times` instead of `times`; this parameter (and, equivalently, `compute_phases`) allows us to override the values contained in the `times` parameter. You should think of `times` as the times of observation that correspond to the actual data -- and they must have the same length as `fluxes`, if provided; `compute_times`, on the other hand, are the timestamps in which we want our model computed. Similarly, `solver_times` determine the timestamp array to be used in the inverse problem. More on that later in the week.

Next, `context='compute'` parameters:

In [5]:
b.filter(dataset='lc01', context='compute').qualifiers

['enabled', 'fti_method']

These parameters determine whether the dataset should be enabled or disabled, and how to handle [finite times of integration](http://phoebe-project.org/docs/2.4/tutorials/fti).

## Radial Velocities

Now let us do the same for a [radial velocity (rv) dataset](http://phoebe-project.org/docs/2.4/tutorials/RV.ipynb):

In [6]:
b.add_dataset('rv', compute_times=phoebe.linspace(0, 1, 11), dataset='rv01')

<ParameterSet: 83 parameters | contexts: figure, constraint, dataset, compute>

Using `dataset='rv01'` will filter on all relevant parameters for the RV dataset:

In [7]:
b.filter(dataset='rv01', context='dataset').qualifiers

['compute_times',
 'compute_phases',
 'phases_t0',
 'mask_enabled',
 'mask_phases',
 'solver_times',
 'passband',
 'intens_weighting',
 'times',
 'rv_offset',
 'ld_mode']

Here we see again parameters describing observables (`times`, `rvs`, `sigmas`), passband information (`passband`), limb darkening (`ld_mode`), instrumental information (`intens_weighting`), phasing (`phases_t0`), data masking (`mask_enabled`, `mask_phases`) and model timestamps (`compute_times`, `compute_phases`, `solver_times`). Some of the parameters we saw in the 'lc' dataset are not applicable when computing rvs, so they are not included (`l3_mode`, `pblum_mode`, etc).

Currently, finite exposure times are not supported for radial velocities in PHOEBE, hence no `exptime` parameter.

Note that, unlike a single `compute_times` parameter, there are **2** `times` parameters - one for each star.  When passing `times=np.linspace(...)` to `b.add_dataset`, both `times` parameters are set to that value.  We could have also set separate times for the two components by setting the values of `times@primary` and `times@secondary`. Alternatively, you can also pass a dictionary when creating the dataset: `b.add_dataset('rv', times={'primary': [0,1,2], 'secondary': [1,2,3]})`.

In [9]:
b.filter(dataset='rv01', context='dataset', qualifier='times').components

['primary', 'secondary']

We can now take a look at the added parameters with context='compute':

In [10]:
b.filter(dataset='rv01', context='compute').qualifiers

['enabled', 'rv_method', 'rv_grav']

Here again we see the `enabled` parameter, the method for computing radial velocities (`rv_method`), and an option to enable/disable gravitational redshift when computing RVs (`rv_grav`).

## Orbits

Although it is not common to have astrometric observations of an orbit, the [orb dataset](http://phoebe-project.org/docs/2.4/tutorials/ORB.ipynb) allows you to synthesize orbits of the stars in a system at any given time, which can be useful for visualization.

In [11]:
b.add_dataset('orb', compute_times=phoebe.linspace(0, 1, 101), dataset='orb01')

<ParameterSet: 50 parameters | contexts: figure, constraint, dataset, compute>

Orbits are parametrized by providing the compute times/phases and enabling/disabling. Note that there is no `times` parameter, as there are no observations. However, if you pass `times` to `add_dataset`, they will be adopted automatically as `compute_times`.

In [12]:
b.filter(dataset='orb01', context='dataset').qualifiers

['compute_times', 'compute_phases', 'phases_t0']

In [13]:
b.filter(dataset='orb01', context='compute').qualifiers

['enabled']

## Line Profiles

[Line profiles](http://phoebe-project.org/docs/2.4/tutorials/LP) are time- *and* wavelength-dependent.  Note that the times cannot be changed after the dataset has been created and attached to the bundle while the wavelengths can. That said, `compute_times` can be changed and used to synthesize line profiles at any desired timestamp. Because of this, the distinction between `times` and `compute_times` is a bit more important for line profiles - you probably only want to provide `times` if you want to attach your actual observations, otherwise just provide `compute_times`.

In [14]:
b.add_dataset('lp', 
              compute_times=phoebe.linspace(0, 1, 11), 
              wavelengths=phoebe.linspace(549, 551, 101), 
              dataset='lp01')

<ParameterSet: 64 parameters | contexts: figure, constraint, dataset, compute>

Now we can take a look at the parameters for a line profile dataset with `context='dataset'`:

In [15]:
b.filter(dataset='lp01', context='dataset').qualifiers

['compute_times',
 'compute_phases',
 'phases_t0',
 'solver_times',
 'profile_func',
 'profile_rest',
 'profile_sv',
 'passband',
 'intens_weighting',
 'wavelengths',
 'ld_mode']

Line profiles are parametrized differently than LCs or RVs; we need to provide a generic profile function (`profile_func`) along with its rest wavelength (`profile_rest`) and subsidiary variable (`profile_sv`), defined as $sv = (p-p_0)/(0.5 \times \mathrm{FWHM})$, where $p_0$ is the rest wavelength and $\mathrm{FWHM}$ is full width at half maximum. Wavelengths are passed as arrays, either to the binary as a whole, or to individual stars.

As far as `context='compute'` parameters are concerned, we only have the enabled/disabled switch:

In [17]:
b.filter(dataset='lp01', context='compute').qualifiers

['enabled']

## Meshes

Similarly to orbits, the [mesh dataset](http://phoebe-project.org/docs/2.4/tutorials/MESH.ipynb) is unlikely to be compared to data directly, but is very useful for visualization of the system. This dataset takes a list of times, columns, and coordinates at which you would like the meshes used within PHOEBE exposed.

As orbits, meshes do not have associated observables, thus they are parametrized only by `compute_times`. As before, passing `times` will automatically be converted to `compute_times` under the hood.

In [18]:
b.add_dataset('mesh', 
              compute_times=[0, 0.5, 1], 
              dataset='mesh01')

<ParameterSet: 85 parameters | contexts: figure, constraint, dataset, compute>

Let us take a look at parameters in the 'dataset' and 'compute' contexts:

In [19]:
b.filter(dataset='mesh01', context='dataset').qualifiers

['compute_times',
 'compute_phases',
 'phases_t0',
 'include_times',
 'coordinates',
 'columns']

In [20]:
b.filter(dataset='mesh01', context='compute').qualifiers

['enabled']

A unique aspect of parametrizing meshes is the `columns` parameter; we will revisit that in great detail soon, but at this point let us take a quick look at the available columns. The list is quite extensive and we will not go into any details at this point, but it should be informative to glance through the names of the columns; all these are per-surface-element values that can be exposed and visualized.

In [21]:
b.get_parameter(dataset='mesh01', context='dataset', qualifier='columns').choices

['volume',
 'xs',
 'ys',
 'zs',
 'vxs',
 'vys',
 'vzs',
 'nxs',
 'nys',
 'nzs',
 'us',
 'vs',
 'ws',
 'vus',
 'vvs',
 'vws',
 'nus',
 'nvs',
 'nws',
 'areas',
 'loggs',
 'teffs',
 'rprojs',
 'mus',
 'visibilities',
 'visible_centroids',
 'rs',
 'intensities@lc01',
 'normal_intensities@lc01',
 'abs_intensities@lc01',
 'abs_normal_intensities@lc01',
 'boost_factors@lc01',
 'ldint@lc01',
 'pblum_ext@lc01',
 'abs_pblum_ext@lc01',
 'ptfarea@lc01',
 'abs_intensities@rv01',
 'abs_normal_intensities@rv01',
 'boost_factors@rv01',
 'ldint@rv01',
 'abs_pblum_ext@rv01',
 'ptfarea@rv01',
 'rvs@rv01',
 'abs_intensities@lp01',
 'abs_normal_intensities@lp01',
 'boost_factors@lp01',
 'ldint@lp01',
 'abs_pblum_ext@lp01',
 'ptfarea@lp01',
 'rvs@lp01']

# Exercise

What choices are available for `rv_method`, `passband`, `pblum_mode`, `ld_mode`, and `intens_weighting`?

Say we have a single-lined binary system. Set the RV dataset times so that only primary star observables are computed.

Add another RV dataset.  Set the new RV dataset to have `rv_method='dynamical'` while keeping the original 'rv01' dataset with `rv_method='flux-weighted'`. This is another example of a per-component parameter: you will either need to set parameters for both components in turn, or use [set_value_all()](http://phoebe-project.org/docs/latest/api/phoebe.parameters.ParameterSet.set_value_all.md).